# VAME tutorial

In this tutorial we'll go through an entire VAME pipeline, step-by-step, including:
- loading the data
- preprocessing the data
- training the VAME model
- evaluating the VAME model
- segmenting the behavior into motifs
- clustering the motifs into communities
- visualizing the results

Let's start by importing the necessary libraries:

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import vame
from vame.util.sample_data import download_sample_data
from pathlib import Path

## Input data

To quickly try VAME, you can download sample data and use it as input. If you want to work with your own data, all you need to do is to provide the paths to the pose estimation files as lists of strings. You can also optionally provide the paths to the corresponding video files.

In [ ]:
# You can run VAME with data from different sources:
# "DeepLabCut", "SLEAP" or "LightningPose"
source_software = "DeepLabCut"

# Download sample data
ps = download_sample_data(source_software)
videos = [ps["video"]]
poses_estimations = [ps["poses"]]

print(videos)
print(poses_estimations)

## Step 1: Initialize your project

In [ ]:
config_file, config_data = vame.init_new_project(
    project_name="my_vame_project",
    poses_estimations=poses_estimations,
    source_software="DeepLabCut",
    videos=videos,
    video_type='.mp4'
)

Let's take a look at the project's configuration:

In [ ]:
print(config_data)

Now let's take a look at the input dataset:

In [ ]:
ds_path = Path(config_data["project_path"]) / "data" / "raw" / f"{config_data['session_names'][0]}.nc"
vame.io.load_poses.load_vame_dataset(ds_path)

## Step 2: Preprocess the raw pose estimation data
The preprocessing step includes:
- Cleaning low confidence data points
- Egocentric alignment using key reference points
- Outlier cleaning
- Savitzky-Golay filtering

In [ ]:
vame.preprocessing(
    config=config_data,
    centered_reference_keypoint="snout",
    orientation_reference_keypoint="tailbase",
)

## Step 3: Train the VAME model

We start by splitting the data into train and test sets:

In [ ]:
vame.create_trainset(config=config_data)

Now we can train the VAME model. This migth take a while, depending on dataset size and your hardware.

In [ ]:
vame.train_model(config=config_data)

## Step 4: Evaluate the VAME model

In [ ]:
vame.evaluate_model(config=config_data)

## Step 5: Segmenting the behavior into motifs

In [ ]:
vame.segment_session(config=config_data)

## Step 6: Create behavioural hierarchies via community clustering

In [ ]:
vame.community(
    config=config_data,
    segmentation_algorithm='hmm',
    cut_tree=2,
)

## Step 7: Visualizing the results

In [ ]:
from vame.visualization.motif import visualize_motif_tree
from vame.visualization.umap import visualize_umap
from vame.visualization.preprocessing import (
    visualize_preprocessing_scatter,
    visualize_preprocessing_timeseries,
)
from vame.visualization.model import plot_loss

In [ ]:
visualize_preprocessing_scatter(config=config_data)

In [ ]:
visualize_preprocessing_timeseries(config=config_data)

In [ ]:
plot_loss(cfg=config_data, model_name="VAME")

In [ ]:
visualize_motif_tree(
    config=config_data,
    segmentation_algorithm="hmm",
)

In [ ]:
visualize_umap(
    config=config_data,
    label="community",
    segmentation_algorithm="hmm",
)

---
#### The following are optional choices to create motif videos, communities/hierarchies of behavior and community videos

In [ ]:
# # OPTIONAL: Create motif videos to get insights about the fine grained poses
vame.motif_videos(config, video_type='.mp4', segmentation_algorithm='hmm')

In [ ]:
# # OPTIONAL: Create community videos to get insights about behavior on a hierarchical scale
vame.community_videos(config, segmentation_algorithm='hmm')

In [ ]:
# # OPTIONAL: Use the generative model (reconstruction decoder) to sample from
# # the learned data distribution, reconstruct random real samples or visualize
# # the cluster center for validation
vame.generative_model(config, mode="sampling", segmentation_algorithm='hmm') #options: mode: "sampling", "reconstruction", "centers", "motifs"

In [ ]:
# OPTIONAL: Create a video of an egocentrically aligned mouse + path through
# the community space (similar to our gif on github) to learn more about your representation
# and have something cool to show around ;)
# Note: This function is currently very slow. Once the frames are saved you can create a video
# or gif via e.g. ImageJ or other tools
vame.gif(config, segmentation_algorithm='hmm', pose_ref_index=[0,5], subtract_background=True, start=None,
         length=30, max_lag=30, label='community', file_format='.mp4', crop_size=(300,300))
